In [1]:
import os

import albumentations as A
import albumentations.augmentations.functional as F
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
from statistics import mean
import torchvision.transforms as T
import torch.onnx
import onnx
import onnxruntime
from torchinfo import summary

from utils.lr_scheduler import PolynomialLRDecay
from models.experiments.fast_scnn_mod import FastSCNN as fastscnn_mod

from utils.dataset import PackagesDataset, PackagesInferenceDataset
from utils.evaluation import Evaluate

/Users/giacomozema/opt/miniconda3/envs/ml/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Input dimension summary

### Input resolution

In [9]:
dims = [512, 256, 224, 192, 160, 128, 96]
model = fastscnn_mod(in_channels=3, num_classes=1, a=1, t=6, r=3, pp=False, verbose=False)
for dim in dims:
    input_size = (1, 3, dim, dim)
    print(f"input size = {dim}x{dim}x{3}")
    summary(model, input_size=input_size, verbose=1, depth=0)
    print()

input size = 512x512x3
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          1,122,673
Total params: 1,122,673
Trainable params: 1,122,673
Non-trainable params: 0
Total mult-adds (M): 826.65
Input size (MB): 3.15
Forward/backward pass size (MB): 200.97
Params size (MB): 4.34
Estimated Total Size (MB): 208.45

input size = 256x256x3
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 256, 256]          1,122,673
Total params: 1,122,673
Trainable params: 1,122,673
Non-trainable params: 0
Total mult-adds (M): 206.68
Input size (MB): 0.79
Forward/backward pass size (MB): 50.24
Params size (MB): 4.34
Estimated Total Size (MB): 55.36

input size = 224x224x3
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 224, 224]          1,122,673

### Input channels

In [10]:
model = fastscnn_mod(in_channels=1, num_classes=1, a=1, t=6, r=3, pp=False, verbose=False)
for dim in dims:
    input_size = (1, 1, dim, dim)
    print(f"input size = {dim}x{dim}x{1}")
    summary(model, input_size=input_size, verbose=1, depth=0)
    print()

input size = 512x512x1
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          1,122,097
Total params: 1,122,097
Trainable params: 1,122,097
Non-trainable params: 0
Total mult-adds (M): 788.90
Input size (MB): 1.05
Forward/backward pass size (MB): 200.97
Params size (MB): 4.33
Estimated Total Size (MB): 206.35

input size = 256x256x1
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 256, 256]          1,122,097
Total params: 1,122,097
Trainable params: 1,122,097
Non-trainable params: 0
Total mult-adds (M): 197.24
Input size (MB): 0.26
Forward/backward pass size (MB): 50.24
Params size (MB): 4.33
Estimated Total Size (MB): 54.84

input size = 224x224x1
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 224, 224]          1,122,097

## Network Topology

### Width Multiplication

In [11]:
parameters1 = [
    (1.0, 6, 3, True),
    (0.75, 6, 3, True),
    (0.5, 6, 3, True),
    (0.25, 6, 3, True),
    (0.125, 6, 3, True),
    (0.75, 4, 3, True),
    (0.5, 4, 3, True),
    (0.25, 4, 3, True),
    (0.125, 4, 3, True),
    (0.75, 2, 3, True),
    (0.5, 2, 3, True),
    (0.25, 2, 3, True),
    (0.125, 2, 3, True),
]

parameters2 = [
    (1.0, 6, 3, True),
    (1.0, 6, 3, False),
    (1.0, 6, 2, True),
    (1.0, 6, 2, False),
    (1.0, 6, 1, True),
    (1.0, 6, 1, False),
]

parameters3 = [
    (0.25, 6, 2, False),
    (0.25, 6, 1, False),
    (0.25, 4, 2, False),
    (0.25, 4, 1, False),
    (0.25, 2, 2, False),
    (0.25, 2, 1, False),
    (0.125, 6, 2, False),
    (0.125, 6, 1, False),
    (0.125, 4, 2, False),
    (0.125, 4, 1, False),
    (0.125, 2, 2, False),
    (0.125, 2, 1, False),
]

In [7]:
for a, t, r, pp in parameters1:
    model = fastscnn_mod(in_channels=3, num_classes=1, a=a, t=t, r=r, pp=pp, verbose=False)
    print(f"a={a} t={t} r={r} pp={pp}")
    input_size = (1, 3, 512, 512)
    summary(model, input_size=input_size, verbose=1, depth=0)
    print()

a=1.0 t=6 r=3 pp=True
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          1,122,673
Total params: 1,122,673
Trainable params: 1,122,673
Non-trainable params: 0
Total mult-adds (M): 830.67
Input size (MB): 3.15
Forward/backward pass size (MB): 209.88
Params size (MB): 4.49
Estimated Total Size (MB): 217.52

a=0.75 t=6 r=3 pp=True
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          644,917
Total params: 644,917
Trainable params: 644,917
Non-trainable params: 0
Total mult-adds (M): 485.61
Input size (MB): 3.15
Forward/backward pass size (MB): 157.42
Params size (MB): 2.58
Estimated Total Size (MB): 163.14

a=0.5 t=6 r=3 pp=True
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          298,553
Total p

### Operator Repetition

In [8]:
for a, t, r, pp in parameters2:
    model = fastscnn_mod(in_channels=3, num_classes=1, a=a, t=t, r=r, pp=pp, verbose=False)
    print(f"a={a} t={t} r={r} pp={pp}")
    input_size = (1, 3, 512, 512)
    summary(model, input_size=input_size, verbose=1, depth=0)
    print()

a=1.0 t=6 r=3 pp=True
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          1,122,673
Total params: 1,122,673
Trainable params: 1,122,673
Non-trainable params: 0
Total mult-adds (M): 830.67
Input size (MB): 3.15
Forward/backward pass size (MB): 209.88
Params size (MB): 4.49
Estimated Total Size (MB): 217.52

a=1.0 t=6 r=3 pp=False
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          1,122,673
Total params: 1,122,673
Trainable params: 1,122,673
Non-trainable params: 0
Total mult-adds (M): 826.65
Input size (MB): 3.15
Forward/backward pass size (MB): 200.97
Params size (MB): 4.34
Estimated Total Size (MB): 208.45

a=1.0 t=6 r=2 pp=True
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          743,281
T

### Width Multiplication and Operator Repetition

In [12]:
for a, t, r, pp in parameters3:
    model = fastscnn_mod(in_channels=3, num_classes=1, a=a, t=t, r=r, pp=pp, verbose=False)
    print(f"a={a} t={t} r={r} pp={pp}")
    input_size = (1, 3, 512, 512)
    summary(model, input_size=input_size, verbose=1, depth=0)
    print()

a=0.25 t=6 r=2 pp=False
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          55,549
Total params: 55,549
Trainable params: 55,549
Non-trainable params: 0
Total mult-adds (M): 61.19
Input size (MB): 3.15
Forward/backward pass size (MB): 43.88
Params size (MB): 0.21
Estimated Total Size (MB): 47.23

a=0.25 t=6 r=1 pp=False
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          27,517
Total params: 27,517
Trainable params: 27,517
Non-trainable params: 0
Total mult-adds (M): 51.46
Input size (MB): 3.15
Forward/backward pass size (MB): 37.49
Params size (MB): 0.10
Estimated Total Size (MB): 40.73

a=0.25 t=4 r=2 pp=False
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          39,805
Total params: 39,805


## Input Dimension and Network Topology

In [14]:
parameters = [
    (1.0, 6, 1, False),
    (0.25, 6, 1, False),
    (0.125, 6, 2, False),
    (0.125, 4, 2, False),
    (0.125, 2, 2, False),
    (0.125, 2, 1, False)
]

dims = [512, 256, 192, 160, 128, 96]

In [18]:
for dim in dims:
    for a, t, r, pp in parameters:
        model = fastscnn_mod(in_channels=3, num_classes=1, a=a, t=t, r=r, pp=pp, verbose=False)
        print(f"a={a} t={t} r={r} pp={pp}")
        input_size = (1, 3, dim, dim)
        summary(model, input_size=input_size, verbose=1, depth=0)
        print()

a=1.0 t=6 r=1 pp=False
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          363,889
Total params: 363,889
Trainable params: 363,889
Non-trainable params: 0
Total mult-adds (M): 555.41
Input size (MB): 3.15
Forward/backward pass size (MB): 149.85
Params size (MB): 1.30
Estimated Total Size (MB): 154.30

a=0.25 t=6 r=1 pp=False
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          27,517
Total params: 27,517
Trainable params: 27,517
Non-trainable params: 0
Total mult-adds (M): 51.46
Input size (MB): 3.15
Forward/backward pass size (MB): 37.49
Params size (MB): 0.10
Estimated Total Size (MB): 40.73

a=0.125 t=6 r=2 pp=False
Layer (type:depth-idx)                        Output Shape              Param #
FastSCNN                                      [1, 1, 512, 512]          16,919
Total params: 1